In [1]:
import pandas as pd 
import numpy as np 

In [2]:
LIHC = pd.read_csv('raw_data/HiSeqV2_LIHC.gz', compression='gzip', sep='\t')
BRCA = pd.read_csv('raw_data/HiSeqV2_BRCA.gz', compression='gzip', sep='\t')
STAD = pd.read_csv('raw_data/HiSeqV2_STAD.gz', compression='gzip', sep='\t')
Normal = pd.read_csv('raw_data/HiSeqV2_Normal.gz', compression='gzip', sep='\t')

In [3]:
LIHC.rename(columns={'sample':'Gene_ID'}, inplace=True)
BRCA.rename(columns={'sample':'Gene_ID'}, inplace=True)
STAD.rename(columns={'sample':'Gene_ID'}, inplace=True)
Normal.rename(columns={'sample':'Gene_ID'}, inplace=True)

In [4]:
LIHC_normal = pd.read_csv('raw_data/liver_normal.tsv', sep='\t')
BRCA_normal = pd.read_csv('raw_data/breast_normal.tsv', sep='\t')
STAD_normal = pd.read_csv('raw_data/stomach_normal.tsv', sep='\t')

In [5]:
# get all normal patients 
LIHC_normal = set(LIHC_normal[LIHC_normal['_study'] == str('GTEX')]['sample'])
BRCA_normal = set(BRCA_normal[BRCA_normal['_study'] == str('GTEX')]['sample'])
STAD_normal = set(STAD_normal[STAD_normal['_study'] == str('GTEX')]['sample'])

In [6]:
TCGA_genes = set(STAD['Gene_ID']).intersection(set(LIHC['Gene_ID']).intersection(set(BRCA['Gene_ID'])))

In [7]:
All_genes = set(Normal['Gene_ID']).intersection(TCGA_genes)

In [8]:
LIHC_patients = np.array(LIHC.columns[1:])
BRCA_patients = np.array(BRCA.columns[1:])
STAD_patients = np.array(STAD.columns[1:])

In [9]:
def filter_patients(df, file):
    to_drop = [] 
    for patient in file: 
        if patient[-2:] == '11': 
            try: 
                del df[patient]
            except:
                pass 
    return df

In [10]:
def keep_mutual_genes(df, genes):
    return df[df['Gene_ID'].isin(genes)]

In [11]:
def rename_header(df):
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header #set the header row as the df header
    return df 

In [12]:
def reorder_genes(df, genes):
    df = df[genes]
    return df 

In [13]:
# get all patients ('01')
LIHC = filter_patients(LIHC, LIHC_patients)
BRCA = filter_patients(BRCA, BRCA_patients)
STAD = filter_patients(STAD, STAD_patients)

In [14]:
# get genes of normal patients 
LIHC_normal_genes = Normal[['Gene_ID']+list(LIHC_normal)]
BRCA_normal_genes = Normal[['Gene_ID']+list(BRCA_normal)]
STAD_normal_genes = Normal[['Gene_ID']+list(STAD_normal)]

In [15]:
LIHC = keep_mutual_genes(LIHC, All_genes)
BRCA = keep_mutual_genes(BRCA, All_genes)
STAD = keep_mutual_genes(STAD, All_genes)
LIHC_Normal = keep_mutual_genes(LIHC_normal_genes, All_genes)
BRCA_Normal = keep_mutual_genes(BRCA_normal_genes, All_genes)
STAD_Normal = keep_mutual_genes(STAD_normal_genes, All_genes)

In [16]:
LIHC = LIHC.transpose()
BRCA = BRCA.transpose()
STAD = STAD.transpose()
LIHC_Normal = LIHC_Normal.transpose()
BRCA_Normal = BRCA_Normal.transpose()
STAD_Normal = STAD_Normal.transpose()

In [17]:
LIHC = rename_header(LIHC)
BRCA = rename_header(BRCA)
STAD = rename_header(STAD)
LIHC_Normal = rename_header(LIHC_Normal)
BRCA_Normal = rename_header(BRCA_Normal)
STAD_Normal = rename_header(STAD_Normal)

In [18]:
LIHC = reorder_genes(LIHC, All_genes)
BRCA = reorder_genes(BRCA, All_genes)
STAD = reorder_genes(STAD, All_genes)
LIHC_Normal = reorder_genes(LIHC_Normal, All_genes)
BRCA_Normal = reorder_genes(BRCA_Normal, All_genes)
STAD_Normal = reorder_genes(STAD_Normal, All_genes)

In [19]:
LIHC.head()

Gene_ID,DHRS7C,TAF9B,OR10G2,PLEC,FZR1,PDIA3,OPN1SW,NDUFS4,DSCAM,ACTN1,...,FOXF1,DAPK3,LDB3,SAMM50,SH3PXD2B,DNAH11,POLE2,FAM110C,PAPOLG,FAM69A
TCGA-FV-A495-01,-0.222262,-0.914256,-0.114236,-0.437745,0.758812,0.437319,-1.40986,2.66008,-2.17873,-1.2675,...,-0.437236,0.928328,-1.19705,0.986674,-1.70739,0.364742,0.951916,1.926,-2.70225,0.888514
TCGA-CC-A3MB-01,-0.222262,-0.471856,-0.114236,-0.525045,0.312212,0.639619,0.748736,1.17258,-2.17873,-0.600595,...,-1.08494,-0.0694719,-2.93395,0.949674,-2.12519,0.0762422,1.61842,1.0328,-1.29105,0.136914
TCGA-BC-A3KF-01,-0.222262,-0.416856,-0.114236,-1.05034,0.512012,0.363919,-0.575164,0.531381,-2.17873,-1.2709,...,-2.80994,-0.171772,-2.91485,0.269774,-1.82809,0.102342,0.00941595,-0.885601,-1.48865,-0.240286
TCGA-DD-A4NV-01,-0.222262,-0.223056,0.454564,-0.817045,-0.403588,0.950119,-0.841064,0.480881,-1.20293,-0.581995,...,0.250864,-0.158972,-0.394655,-0.472226,-1.99289,4.31194,-1.09628,1.8146,-1.42895,0.568114
TCGA-CC-A1HT-01,0.123838,-0.865256,-0.114236,0.373755,-0.580788,0.644719,-0.173764,0.0658813,-1.83263,-0.137995,...,-0.510236,-0.0550719,-3.18005,-0.844326,-0.76979,0.315542,0.481216,0.693499,-0.0945477,0.495214


In [20]:
BRCA.head()

Gene_ID,DHRS7C,TAF9B,OR10G2,PLEC,FZR1,PDIA3,OPN1SW,NDUFS4,DSCAM,ACTN1,...,FOXF1,DAPK3,LDB3,SAMM50,SH3PXD2B,DNAH11,POLE2,FAM110C,PAPOLG,FAM69A
TCGA-AR-A5QQ-01,-0.222262,-0.356756,-0.114236,1.46686,0.724212,0.810819,0.583136,0.445081,4.28777,1.9697,...,-2.32964,0.673428,-2.64795,-0.136926,0.54461,0.611742,1.74502,2.5997,-0.427348,1.00831
TCGA-D8-A1JA-01,-0.222262,1.59224,-0.114236,-1.53214,-1.43979,0.755919,-0.551764,-0.964919,-0.943429,-0.157295,...,-3.39394,-1.57567,0.194145,0.0607745,0.46461,-0.188958,0.943416,-0.629201,0.789452,0.601114
TCGA-BH-A0BQ-01,-0.222262,-0.0199564,-0.114236,-0.190745,-0.0182884,0.565219,-1.40986,0.250481,-0.915729,1.1878,...,-2.08194,0.781328,-1.10115,0.310374,1.34071,0.909642,-0.068984,0.619999,-0.0898477,-0.461586
TCGA-BH-A0BT-01,-0.222262,0.467744,-0.114236,-0.529245,-0.732688,0.300719,-0.305264,-0.869419,-0.455829,0.376205,...,-2.78014,-0.741372,-1.50695,-0.315426,-0.48369,-1.92896,1.54992,-1.1809,-0.374248,-0.912886
TCGA-A8-A06X-01,-0.222262,0.312444,-0.114236,0.077555,0.393112,-0.102881,-0.827964,-0.704919,-2.17873,-0.137595,...,-2.22654,0.661028,0.520245,-0.516526,1.46271,-3.84006,0.670516,-0.242201,1.03765,-0.159686


In [21]:
STAD.head()

Gene_ID,DHRS7C,TAF9B,OR10G2,PLEC,FZR1,PDIA3,OPN1SW,NDUFS4,DSCAM,ACTN1,...,FOXF1,DAPK3,LDB3,SAMM50,SH3PXD2B,DNAH11,POLE2,FAM110C,PAPOLG,FAM69A
TCGA-FP-A4BE-01,-0.222262,0.616444,-0.114236,1.16556,0.462512,0.329019,-0.104564,0.212481,-2.17873,0.109605,...,2.02566,0.0436281,-1.28775,-0.511526,0.19061,-2.38406,1.53452,-1.5909,-0.0281477,1.16891
TCGA-BR-8081-01,-0.222262,-0.862856,-0.114236,0.956755,-0.119388,-0.394581,1.15654,0.693181,-2.17873,0.510005,...,2.42326,-0.138372,-0.303355,0.217474,0.56201,-3.39786,2.23482,3.0621,0.593252,0.492614
TCGA-IN-7806-01,-0.222262,-0.0763564,-0.114236,0.868255,-0.209388,-1.28958,0.605136,-0.962019,-2.17873,-0.218895,...,2.30316,-0.459872,2.14845,-0.310426,0.11321,0.233842,0.00561595,3.869,0.321852,0.446114
TCGA-BR-4257-01,-0.222262,-0.645556,-0.114236,0.691855,-0.459988,0.712019,0.715936,-0.205019,-2.17873,0.836605,...,2.75216,-0.487672,-2.71805,0.0690745,1.28681,-3.84006,1.50322,1.3798,0.586252,-0.337386
TCGA-RD-A8N6-01,-0.222262,0.222644,-0.114236,1.30526,0.0835116,0.454219,4.06244,-0.206719,0.142271,1.8201,...,3.72466,0.376828,4.71515,0.0281745,-0.33799,-1.57796,0.395916,2.5073,1.67105,0.531314


In [22]:
LIHC_Normal.head()

Gene_ID,DHRS7C,TAF9B,OR10G2,PLEC,FZR1,PDIA3,OPN1SW,NDUFS4,DSCAM,ACTN1,...,FOXF1,DAPK3,LDB3,SAMM50,SH3PXD2B,DNAH11,POLE2,FAM110C,PAPOLG,FAM69A
GTEX-131XE-0326-SM-5LZVO,0,7.9997,0,13.0109,10.1414,15.6009,1.5735,9.4767,3.6846,13.2806,...,9.0623,10.9877,7.0163,9.8046,8.336,6.6864,6.1328,10.7502,8.9743,8.4162
GTEX-ZF29-2026-SM-4WWB7,0,8.8492,0,13.5604,10.6477,14.7868,2.9537,10.4007,1.2324,12.8812,...,7.1012,10.2885,4.6606,10.6032,8.8064,5.8351,7.2365,9.9535,8.1722,9.6722
GTEX-11UD2-1626-SM-5EQM3,2.2539,9.4283,1.5285,13.4725,10.1778,13.6353,2.7343,9.2918,3.8272,13.5794,...,10.5593,11.6689,6.4545,9.5993,9.7041,3.1686,5.5886,11.2039,9.0361,7.8341
GTEX-UPIC-0926-SM-4IHLV,0,9.0713,0,13.711,11.0801,14.011,0,10.1816,2.2028,12.0298,...,6.8841,9.8023,5.8292,10.5983,8.1853,7.4023,7.9322,9.3234,8.0149,8.7775
GTEX-139UC-1426-SM-5IJGH,2.1649,9.0843,0,12.9384,11.2361,13.3928,0,9.9495,2.6384,12.3139,...,8.4051,10.6156,4.8517,9.6974,8.4417,5.0099,6.1434,11.0268,8.4198,9.1718


In [23]:
BRCA_Normal.head()

Gene_ID,DHRS7C,TAF9B,OR10G2,PLEC,FZR1,PDIA3,OPN1SW,NDUFS4,DSCAM,ACTN1,...,FOXF1,DAPK3,LDB3,SAMM50,SH3PXD2B,DNAH11,POLE2,FAM110C,PAPOLG,FAM69A
GTEX-147F4-2826-SM-5NQBN,0,9.6245,0,13.9045,11.3193,13.578,3.4989,10.2763,2.2823,14.4132,...,6.0314,12.1953,8.1253,10.1626,11.5519,4.1015,6.0314,7.8392,9.0684,8.9985
GTEX-WYJK-1326-SM-3NB2T,0.8794,9.8342,0,14.5909,11.2827,13.0695,2.7819,10.1992,2.378,13.6824,...,5.6351,11.2043,8.7845,10.4987,12.9623,4.6495,6.2104,7.7489,9.3775,8.2662
GTEX-11DXW-0626-SM-5N9ER,2.1392,9.839,1.0942,13.9919,11.0615,13.2518,2.1392,10.2256,2.4699,13.9072,...,8.7859,11.7951,9.3499,10.44,11.549,2.5078,6.7659,7.4763,9.2646,8.8546
GTEX-11P81-1926-SM-5BC53,1.7947,9.6896,0,13.56,10.8489,13.118,0,10.1711,4.0918,13.6121,...,6.2064,10.5213,7.7372,11.0387,12.0531,4.9387,6.2064,8.0777,9.7611,9.2152
GTEX-13NZ9-1026-SM-5MR5K,0,9.1844,0,14.0806,11.0314,13.5734,1.5651,10.0602,0,14.169,...,7.6497,11.3548,5.0584,10.3331,11.1406,2.3599,6.079,7.3965,9.6354,8.2272


In [24]:
STAD_Normal.head()

Gene_ID,DHRS7C,TAF9B,OR10G2,PLEC,FZR1,PDIA3,OPN1SW,NDUFS4,DSCAM,ACTN1,...,FOXF1,DAPK3,LDB3,SAMM50,SH3PXD2B,DNAH11,POLE2,FAM110C,PAPOLG,FAM69A
GTEX-WFG7-1326-SM-4LMK1,0,8.7858,0,13.8329,10.8574,14.1243,2.3842,9.7086,1.2672,12.3178,...,9.4045,10.6894,7.5662,11.0177,10.9237,4.5238,7.3344,8.3673,8.9992,8.1728
GTEX-P4QT-1526-SM-3NMCT,0,10.1828,0,15.0838,10.586,12.7937,3.8027,10.1777,2.6136,16.2197,...,12.9958,12.6573,11.0574,10.0691,13.2748,6.2538,5.0777,6.4263,8.4605,9.2694
GTEX-R55G-1126-SM-48FDG,0,10.665,0,14.1017,10.224,13.5331,1.3631,9.7341,4.0638,15.9184,...,12.1055,11.6771,10.1659,9.9467,13.0208,7.2401,6.0327,5.0883,8.8943,9.7506
GTEX-1122O-1926-SM-5EGIQ,0,8.8462,0,13.8969,10.5078,13.283,2.5074,9.978,1.7411,11.9833,...,9.4544,11.1612,8.2443,11.0055,10.6471,4.6768,6.1074,9.2475,8.8608,8.002
GTEX-13O61-1126-SM-5L3FI,2.3164,10.6534,0,14.0024,10.6343,12.8384,2.6572,10.4699,3.9632,15.3699,...,12.2094,11.9559,10.0279,10.2548,11.952,7.6451,4.7122,4.7834,8.9307,9.8635


In [25]:
LIHC = LIHC.astype(float)
BRCA = BRCA.astype(float)
STAD = STAD.astype(float)
LIHC_Normal = LIHC_Normal.astype(float)
BRCA_Normal = BRCA_Normal.astype(float)
STAD_Normal = STAD_Normal.astype(float)

In [26]:
LIHC.to_csv('data/LIHC.csv') 
BRCA.to_csv('data/BRCA.csv') 
STAD.to_csv('data/STAD.csv') 
LIHC_Normal.to_csv('data/LIHC_Normal.csv') 
BRCA_Normal.to_csv('data/BRCA_Normal.csv') 
STAD_Normal.to_csv('data/STAD_Normal.csv') 

In [27]:
LIHC = pd.read_csv('data/LIHC.csv')
BRCA = pd.read_csv('data/BRCA.csv')
STAD = pd.read_csv('data/STAD.csv')
LIHC_Normal = pd.read_csv('data/LIHC_Normal.csv')
BRCA_Normal = pd.read_csv('data/BRCA_Normal.csv')
STAD_Normal = pd.read_csv('data/STAD_Normal.csv')I 

In [28]:
LIHC.head()

,Unnamed: 0,DHRS7C,TAF9B,OR10G2,PLEC,FZR1,PDIA3,OPN1SW,NDUFS4,DSCAM,...,FOXF1,DAPK3,LDB3,SAMM50,SH3PXD2B,DNAH11,POLE2,FAM110C,PAPOLG,FAM69A
0,TCGA-FV-A495-01,-0.222262,-0.914256,-0.114236,-0.437745,0.758812,0.437319,-1.409864,2.660081,-2.178729,...,-0.437236,0.928328,-1.197055,0.986674,-1.70739,0.364742,0.951916,1.925999,-2.702248,0.888514
1,TCGA-CC-A3MB-01,-0.222262,-0.471856,-0.114236,-0.525045,0.312212,0.639619,0.748736,1.172581,-2.178729,...,-1.084936,-0.069472,-2.933955,0.949674,-2.12519,0.076242,1.618416,1.032799,-1.291048,0.136914
2,TCGA-BC-A3KF-01,-0.222262,-0.416856,-0.114236,-1.050345,0.512012,0.363919,-0.575164,0.531381,-2.178729,...,-2.809936,-0.171772,-2.914855,0.269774,-1.82809,0.102342,0.009416,-0.885601,-1.488648,-0.240286
3,TCGA-DD-A4NV-01,-0.222262,-0.223056,0.454564,-0.817045,-0.403588,0.950119,-0.841064,0.480881,-1.202929,...,0.250864,-0.158972,-0.394655,-0.472226,-1.99289,4.311942,-1.096284,1.814599,-1.428948,0.568114
4,TCGA-CC-A1HT-01,0.123838,-0.865256,-0.114236,0.373755,-0.580788,0.644719,-0.173764,0.065881,-1.832629,...,-0.510236,-0.055072,-3.180055,-0.844326,-0.76979,0.315542,0.481216,0.693499,-0.094548,0.495214


In [29]:
BRCA.head()

,Unnamed: 0,DHRS7C,TAF9B,OR10G2,PLEC,FZR1,PDIA3,OPN1SW,NDUFS4,DSCAM,...,FOXF1,DAPK3,LDB3,SAMM50,SH3PXD2B,DNAH11,POLE2,FAM110C,PAPOLG,FAM69A
0,TCGA-AR-A5QQ-01,-0.222262,-0.356756,-0.114236,1.466855,0.724212,0.810819,0.583136,0.445081,4.287771,...,-2.329636,0.673428,-2.647955,-0.136926,0.54461,0.611742,1.745016,2.599699,-0.427348,1.008314
1,TCGA-D8-A1JA-01,-0.222262,1.592244,-0.114236,-1.532145,-1.439788,0.755919,-0.551764,-0.964919,-0.943429,...,-3.393936,-1.575672,0.194145,0.060774,0.46461,-0.188958,0.943416,-0.629201,0.789452,0.601114
2,TCGA-BH-A0BQ-01,-0.222262,-0.019956,-0.114236,-0.190745,-0.018288,0.565219,-1.409864,0.250481,-0.915729,...,-2.081936,0.781328,-1.101155,0.310374,1.34071,0.909642,-0.068984,0.619999,-0.089848,-0.461586
3,TCGA-BH-A0BT-01,-0.222262,0.467744,-0.114236,-0.529245,-0.732688,0.300719,-0.305264,-0.869419,-0.455829,...,-2.780136,-0.741372,-1.506955,-0.315426,-0.48369,-1.928958,1.549916,-1.180901,-0.374248,-0.912886
4,TCGA-A8-A06X-01,-0.222262,0.312444,-0.114236,0.077555,0.393112,-0.102881,-0.827964,-0.704919,-2.178729,...,-2.226536,0.661028,0.520245,-0.516526,1.46271,-3.840058,0.670516,-0.242201,1.037652,-0.159686


In [30]:
STAD.head()

,Unnamed: 0,DHRS7C,TAF9B,OR10G2,PLEC,FZR1,PDIA3,OPN1SW,NDUFS4,DSCAM,...,FOXF1,DAPK3,LDB3,SAMM50,SH3PXD2B,DNAH11,POLE2,FAM110C,PAPOLG,FAM69A
0,TCGA-FP-A4BE-01,-0.222262,0.616444,-0.114236,1.165555,0.462512,0.329019,-0.104564,0.212481,-2.178729,...,2.025664,0.043628,-1.287755,-0.511526,0.19061,-2.384058,1.534516,-1.590901,-0.028148,1.168914
1,TCGA-BR-8081-01,-0.222262,-0.862856,-0.114236,0.956755,-0.119388,-0.394581,1.156536,0.693181,-2.178729,...,2.423264,-0.138372,-0.303355,0.217474,0.56201,-3.397858,2.234816,3.062099,0.593252,0.492614
2,TCGA-IN-7806-01,-0.222262,-0.076356,-0.114236,0.868255,-0.209388,-1.289581,0.605136,-0.962019,-2.178729,...,2.303164,-0.459872,2.148445,-0.310426,0.11321,0.233842,0.005616,3.868999,0.321852,0.446114
3,TCGA-BR-4257-01,-0.222262,-0.645556,-0.114236,0.691855,-0.459988,0.712019,0.715936,-0.205019,-2.178729,...,2.752164,-0.487672,-2.718055,0.069074,1.28681,-3.840058,1.503216,1.379799,0.586252,-0.337386
4,TCGA-RD-A8N6-01,-0.222262,0.222644,-0.114236,1.305255,0.083512,0.454219,4.062436,-0.206719,0.142271,...,3.724664,0.376828,4.715145,0.028174,-0.33799,-1.577958,0.395916,2.507299,1.671052,0.531314


In [31]:
LIHC_Normal.head()

,Unnamed: 0,DHRS7C,TAF9B,OR10G2,PLEC,FZR1,PDIA3,OPN1SW,NDUFS4,DSCAM,...,FOXF1,DAPK3,LDB3,SAMM50,SH3PXD2B,DNAH11,POLE2,FAM110C,PAPOLG,FAM69A
0,GTEX-131XE-0326-SM-5LZVO,0.0000,7.9997,0.0000,13.0109,10.1414,15.6009,1.5735,9.4767,3.6846,...,9.0623,10.9877,7.0163,9.8046,8.3360,6.6864,6.1328,10.7502,8.9743,8.4162
1,GTEX-ZF29-2026-SM-4WWB7,0.0000,8.8492,0.0000,13.5604,10.6477,14.7868,2.9537,10.4007,1.2324,...,7.1012,10.2885,4.6606,10.6032,8.8064,5.8351,7.2365,9.9535,8.1722,9.6722
2,GTEX-11UD2-1626-SM-5EQM3,2.2539,9.4283,1.5285,13.4725,10.1778,13.6353,2.7343,9.2918,3.8272,...,10.5593,11.6689,6.4545,9.5993,9.7041,3.1686,5.5886,11.2039,9.0361,7.8341
3,GTEX-UPIC-0926-SM-4IHLV,0.0000,9.0713,0.0000,13.7110,11.0801,14.0110,0.0000,10.1816,2.2028,...,6.8841,9.8023,5.8292,10.5983,8.1853,7.4023,7.9322,9.3234,8.0149,8.7775
4,GTEX-139UC-1426-SM-5IJGH,2.1649,9.0843,0.0000,12.9384,11.2361,13.3928,0.0000,9.9495,2.6384,...,8.4051,10.6156,4.8517,9.6974,8.4417,5.0099,6.1434,11.0268,8.4198,9.1718


In [32]:
BRCA_Normal.head()

,Unnamed: 0,DHRS7C,TAF9B,OR10G2,PLEC,FZR1,PDIA3,OPN1SW,NDUFS4,DSCAM,...,FOXF1,DAPK3,LDB3,SAMM50,SH3PXD2B,DNAH11,POLE2,FAM110C,PAPOLG,FAM69A
0,GTEX-147F4-2826-SM-5NQBN,0.0000,9.6245,0.0000,13.9045,11.3193,13.5780,3.4989,10.2763,2.2823,...,6.0314,12.1953,8.1253,10.1626,11.5519,4.1015,6.0314,7.8392,9.0684,8.9985
1,GTEX-WYJK-1326-SM-3NB2T,0.8794,9.8342,0.0000,14.5909,11.2827,13.0695,2.7819,10.1992,2.3780,...,5.6351,11.2043,8.7845,10.4987,12.9623,4.6495,6.2104,7.7489,9.3775,8.2662
2,GTEX-11DXW-0626-SM-5N9ER,2.1392,9.8390,1.0942,13.9919,11.0615,13.2518,2.1392,10.2256,2.4699,...,8.7859,11.7951,9.3499,10.4400,11.5490,2.5078,6.7659,7.4763,9.2646,8.8546
3,GTEX-11P81-1926-SM-5BC53,1.7947,9.6896,0.0000,13.5600,10.8489,13.1180,0.0000,10.1711,4.0918,...,6.2064,10.5213,7.7372,11.0387,12.0531,4.9387,6.2064,8.0777,9.7611,9.2152
4,GTEX-13NZ9-1026-SM-5MR5K,0.0000,9.1844,0.0000,14.0806,11.0314,13.5734,1.5651,10.0602,0.0000,...,7.6497,11.3548,5.0584,10.3331,11.1406,2.3599,6.0790,7.3965,9.6354,8.2272


In [33]:
STAD_Normal.head()

,Unnamed: 0,DHRS7C,TAF9B,OR10G2,PLEC,FZR1,PDIA3,OPN1SW,NDUFS4,DSCAM,...,FOXF1,DAPK3,LDB3,SAMM50,SH3PXD2B,DNAH11,POLE2,FAM110C,PAPOLG,FAM69A
0,GTEX-WFG7-1326-SM-4LMK1,0.0000,8.7858,0.0,13.8329,10.8574,14.1243,2.3842,9.7086,1.2672,...,9.4045,10.6894,7.5662,11.0177,10.9237,4.5238,7.3344,8.3673,8.9992,8.1728
1,GTEX-P4QT-1526-SM-3NMCT,0.0000,10.1828,0.0,15.0838,10.5860,12.7937,3.8027,10.1777,2.6136,...,12.9958,12.6573,11.0574,10.0691,13.2748,6.2538,5.0777,6.4263,8.4605,9.2694
2,GTEX-R55G-1126-SM-48FDG,0.0000,10.6650,0.0,14.1017,10.2240,13.5331,1.3631,9.7341,4.0638,...,12.1055,11.6771,10.1659,9.9467,13.0208,7.2401,6.0327,5.0883,8.8943,9.7506
3,GTEX-1122O-1926-SM-5EGIQ,0.0000,8.8462,0.0,13.8969,10.5078,13.2830,2.5074,9.9780,1.7411,...,9.4544,11.1612,8.2443,11.0055,10.6471,4.6768,6.1074,9.2475,8.8608,8.0020
4,GTEX-13O61-1126-SM-5L3FI,2.3164,10.6534,0.0,14.0024,10.6343,12.8384,2.6572,10.4699,3.9632,...,12.2094,11.9559,10.0279,10.2548,11.9520,7.6451,4.7122,4.7834,8.9307,9.8635
